In [ ]:
import logging

import anndata2ri
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
import scanpy as sc
import seaborn as sns
from scipy.stats import median_abs_deviation

In [ ]:
adata = sc.read_10x_mtx(
    "../HIPSDR-seq/1_LFS_HIPSDR_GEX_2/outs/filtered_feature_bc_matrix/"
)
adata.var_names_make_unique()

In [ ]:
adata.var["MT"] = adata.var.index.str.startswith("MT-")
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["MT"], percent_top=[20], log1p=True, inplace=True
)
sc.pp.filter_cells(adata, min_genes=200)

In [ ]:
p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
p2 = sc.pl.violin(adata, "pct_counts_MT")
p3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_MT")

In [ ]:
sc.pl.violin(
    adata=adata,
    keys=["n_genes_by_counts", "total_counts", "pct_counts_MT"],
    multi_panel=True,
)

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [ ]:
adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
adata.obs.outlier.value_counts()

In [ ]:
adata_raw = adata.copy()

In [ ]:
adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_MT", 3) | (
    adata.obs["pct_counts_MT"] > 6
)
adata.obs.mt_outlier.value_counts()
adata.layers["counts"] = adata.X.copy()

In [ ]:
print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")

## Ambient RNA removal

In [ ]:
rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
%%R
library(SoupX)

In [ ]:
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp)
sc.pp.log1p(adata_pp)

sc.pp.pca(adata_pp)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="soupx_groups")

# Preprocess variables for SoupX
soupx_groups = adata_pp.obs["soupx_groups"]

del adata_pp

cells = adata.obs_names
genes = adata.var_names
data = adata.X.T

adata_raw = sc.read_10x_h5(
    filename=f"../HIPSDR-seq/1_LFS_HIPSDR_GEX_2/outs/raw_feature_bc_matrix.h5",
)
adata_raw.var_names_make_unique()
data_tod = adata_raw.X.T

del adata_raw

In [ ]:
%%R -o out -i data -i data_tod -i genes -i cells -i soupx_groups 

set.seed(123)
# specify row and column names of data
rownames(data) = genes
colnames(data) = cells
# ensure correct sparse format for table of counts and table of droplets
data <- as(data, "sparseMatrix")
data_tod <- as(data_tod, "sparseMatrix")

# Generate SoupChannel Object for SoupX 
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

# Add extra meta data to the SoupChannel object
soupProf = data.frame(row.names = rownames(data), est = rowSums(data)/sum(data), counts = rowSums(data))
sc = setSoupProfile(sc, soupProf)
# Set cluster information in SoupChannel
sc = setClusters(sc, soupx_groups)

# Estimate contamination fraction
sc  = autoEstCont(sc, doPlot=FALSE)
# Infer corrected table of counts and rount to integer
out = adjustCounts(sc, roundToInt = TRUE)

In [ ]:
out = %Rget out

In [ ]:
adata.layers["counts"] = adata.X.copy()
adata.layers["soupX_counts"] = out.T
adata.X = adata.layers["soupX_counts"].copy()

In [ ]:
print(f"Total number of genes: {adata.n_vars}")

# Min 3 cells - filters out 0 count genes
sc.pp.filter_genes(adata, min_cells=3)
print(f"Number of genes after cell filter: {adata.n_vars}")

In [ ]:
p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
# sc.pl.violin(adata, 'total_counts')
p2 = sc.pl.violin(adata, "pct_counts_MT")
p3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_MT")

In [ ]:
adata_ambient = adata.copy()

## Doublet removal

In [ ]:
%%R
library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)

In [ ]:
data_mat = adata.X.T.copy()

In [ ]:
%%R -i data_mat -o doublet_score -o doublet_class

set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

In [ ]:
doublet_scr = %Rget doublet_score
doublet_cls = %Rget doublet_class

In [ ]:
adata.obs["scDblFinder_score"] = doublet_scr
adata.obs["scDblFinder_class"] = doublet_cls
adata.obs.scDblFinder_class.value_counts()

In [ ]:
adata_doublets = adata.copy()

In [ ]:
print(adata_doublets.X.max())

In [ ]:
adata_doublets.X = adata_doublets.layers["soupX_counts"].copy()
sc.pp.normalize_total(adata_doublets, target_sum=1e4)
sc.pp.log1p(adata_doublets)
sc.pp.highly_variable_genes(adata_doublets, flavor="seurat", n_top_genes=2000)
sc.tl.pca(adata_doublets, svd_solver="arpack", use_highly_variable=True)
sc.pp.neighbors(adata_doublets)
sc.tl.umap(adata_doublets)
sc.pl.umap(adata_doublets, color=["scDblFinder_score", "scDblFinder_class"])

In [ ]:
adata.write_h5ad("hipsdr_rna_qc.h5ad")

## Normalization and clustering

In [ ]:
adata = sc.read_h5ad("../HIPSDR-seq/hipsdr_rna_qc.h5ad")

In [ ]:
adata = adata[adata.obs["scDblFinder_class"] == "singlet"].copy()

In [ ]:
clones = pd.read_csv("../HIPSDR-seq/clusters_final.csv", index_col=0)

In [ ]:
adata = adata[adata.obs_names.isin(clones.index)].copy()
adata

In [ ]:
adata.X = adata.layers["soupX_counts"].copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
adata.layers["logcounts"] = adata.X.copy()
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)
sc.tl.pca(adata, svd_solver="arpack", use_highly_variable=True)
sc.pl.pca_variance_ratio(adata)
sc.pp.neighbors(adata, n_pcs=15)
sc.tl.umap(adata)

In [ ]:
adata.obs["cluster"] = adata.obs_names.map(clones.clusters)

In [ ]:
sc.pl.umap(
    adata,
    color=["n_genes_by_counts", "log1p_total_counts", "pct_counts_MT", "cluster"],
    ncols=2,
)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby="cluster", method="wilcoxon")

In [ ]:
sc.tl.filter_rank_genes_groups(adata)

In [ ]:
plt.figure(figsize=(10, 5))
sc.pl.rank_genes_groups_dotplot(adata, key="rank_genes_groups_filtered", show=False)
plt.tight_layout()
plt.savefig("figures_rebuttal/dotplot_expression.png", bbox_inches="tight", dpi=300)
plt.savefig("figures_rebuttal/dotplot_expression.svg", bbox_inches="tight", dpi=300)

In [ ]:
sc.pl.umap(
    adata,
    color=["cluster"],
    title=["Cluster"],
    ncols=2,
    show=False,
    wspace=0.2,
    palette={
        "Cluster 1": sns.palettes.color_palette("tab10")[6],
        "Cluster 0": sns.palettes.color_palette("tab10")[0],
    },
)
plt.tight_layout()
plt.savefig("figures_rebuttal/scRNA_dna_cluster.png", dpi=300)
plt.savefig("figures_rebuttal/scRNA_dna_cluster.svg", dpi=300)

## Clusters for HIPSD

In [ ]:
cna = pd.read_csv("../data/CNVs_HIPSDR_filtered.csv.gz", index_col=0)
cna

In [ ]:
cna = (cna - 2) / 3

In [ ]:
adata = sc.AnnData(cna)

In [ ]:
adata.var["chromosome"] = adata.var.index.str.split(":").str[0]
adata.var["start"] = (
    adata.var.index.str.split(":").str[1].str.split("-").str[0].astype(int)
)
adata.var["end"] = (
    adata.var.index.str.split(":").str[1].str.split("-").str[1].astype(int)
)

In [ ]:
adata.var.columns = ["chromosome", "start", "end"]
adata.obsm["X_cnv"] = adata.X
adata.var["pos"] = np.arange(adata.var.shape[0])
chrom_dict = {}
chrom_dict["chr_pos"] = {}
for tup in adata.var.itertuples():
    if tup.chromosome not in chrom_dict["chr_pos"]:
        chrom_dict["chr_pos"][tup.chromosome] = tup.pos
    if chrom_dict["chr_pos"][tup.chromosome] > tup.pos:
        chrom_dict["chr_pos"][tup.chromosome] = tup.pos
adata.uns["cnv"] = chrom_dict

In [ ]:
sc.tl.pca(adata, svd_solver="arpack")
sc.pp.neighbors(adata)
sc.tl.leiden(adata, key_added="cnv_leiden", resolution=0.7)
sc.tl.paga(adata, groups="cnv_leiden")
sc.pl.paga(
    adata, plot=True
)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata, init_pos="paga")

In [ ]:
adata.obs["cluster"] = adata.obs.index.map(clones.cluster)

In [ ]:
sc.pl.umap(
    adata,
    color=["cluster"],
    title=["Cluster"],
    palette={
        "Cluster 1": sns.palettes.color_palette("tab10")[6],
        "Cluster 0": sns.palettes.color_palette("tab10")[0],
    },
    ncols=2,
    vmax="p99",
    show=False,
    wspace=0.2,
)
plt.tight_layout()
plt.savefig("figures_rebuttal/HIPSD_DNA_UMAP_clusters.png", dpi=300)